In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import m2lib22.box as box
import m2lib22.sat as sat
import m2lib22.cstes as cstes
import m2lib22.diagnosis as diag

In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(walltime='04:00:00')
    w = cluster.scale(jobs=2)
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.146:8787/status,
Dashboard: http://10.148.0.146:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.146:44374,Workers: 0
Dashboard: http://10.148.0.146:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
labels = cstes.labels

In [4]:
zarr_dir = cstes.zarr_dir
print(zarr_dir)

/home1/datawork/mdemol/m2


In [5]:
DS= {l: xr.open_zarr(cstes.zarr_dir+'/'+l+'.zarr').persist() for l in labels[:-1]}### TO CHANGE
acc_bins=np.arange(-1e-4,1e-4,2e-6)
dist_bins=np.arange(0,1e5,2e3)
diag_list = [diag.diag_ind_pdfs, diag.diag_combined_pdfs]
bins_list = [ {"acc": acc_bins, "alti___distance" : dist_bins} ]*2
                

ds_diag_dist = xr.concat([diag.diags(DS[key].isel( site_obs = DS[key].__site_matchup_indice.compute() )
                                .isel(alti_time=10)
                                .isel(alti_time_mid=10),
                                diag_list, bins_list) for key in DS],
                         dim=pd.Index(DS.keys(),name='drifter_sat_year'))

zarr = os.path.join(zarr_dir, "acc_dist_pdfs.zarr")
ds_diag_dist.to_zarr(zarr, mode="w") 
print('acc_dist_pdfs storred in zarr')

acc_dist_pdfs storred in zarr


In [ ]:
#ds = xr.open_zarr(cstes.zarr_dir+'/'+'acc_dist_pdfs.zarr').persist()

In [ ]:
#ds